# Parte Pablo

In [92]:
irreductible = "0000000110001011"

In [93]:
def polinomio(n):
    binario = bin(n)[2:]
    binario = '0'*16 + binario
    binario = binario[-16:]
    return binario

In [94]:
def grado(n):
    for index, element in enumerate(n):
        if element == '1':
            return 15 - index
    return 0

In [95]:
def shift(n, a):
    n = int(n, 2)
    n = n << a
    return polinomio(n)

In [96]:
def xor(m, n):
    m = int(m, 2)
    n = int(n, 2)
    return polinomio(m ^ n)

In [97]:
def divisible(m, n):
    m = polinomio(m)
    n = polinomio(n)
    
    gradom = grado(m)
    gradon = grado(n)
    
    while gradom >= gradon:
        dif = gradom - gradon
        m = xor(shift(n, dif), m)
        gradom = grado(m)
        
    return int(m, 2) == 0

In [98]:
def irreductible_polynomials():
    irreductibles = []
    for i in range(2, 512):
        if len(irreductibles) == 0:
            irreductibles.append(i)
        else:
            primo = True
            for j in irreductibles:
                primo = not divisible(i, j)
                if not primo:
                    break
            if primo:
                irreductibles.append(i)
    return irreductibles

In [99]:
def grados(n):
    grados = []
    for index, element in enumerate(n):
        if element == "1":
            grados.append(15 - index)
    return grados

In [100]:
def division(m, n):
    gradom = grado(m)
    gradon = grado(n)
    
    while gradom >= gradon:
        dif = gradom - gradon
        m = xor(shift(n, dif), m)
        gradom = grado(m)
        
    return m

In [101]:
def GF_product_p(a, b):
    a = polinomio(a)
    b = polinomio(b)
    gradosl = grados(b)
    polinomiosl = []
    for grado in gradosl:
        if grado == 0:
            polinomiosl.append(a)
        else:
            polinomiosl.append(shift(a, grado))
    
    result = None
    for i in polinomiosl:
        if result is None:
            result = i
        else:
            result = xor(result, i)
    
    if result is None:
        return 0
    
    else:
        result = division(result, irreductible)
        return int(result, 2)

In [102]:
def GF_es_generador(a):
    tabla = set()
    tabla.add(1)
    anterior = 1
    for i in range(1, 255):
        anterior = GF_product_p(anterior, a)
        tabla.add(anterior)
    else:
        return len(tabla) == 255

In [103]:
generadores = []
for i in range(2, 255):
    if GF_es_generador(i):
        generadores.append(i)

In [104]:
def GF_tables():
    exponencial = [1] * 255
    logaritmica = [0] * 256
    for i in range(1, 255):
        result = GF_product_p(exponencial[i - 1], 6)
        logaritmica[result] = i
        exponencial[i] = result

In [105]:
def potencia(a, e):
    result = 1
    for i in range(0, e):
        result = GF_product_p(result, a)
    return result


In [106]:
def GF_product_t(a, b):
    if a == 0 or b == 0:
        return 0
    exponente1 = logaritmica[a]
    exponente2 = logaritmica[b]
    exponenteT = (exponente1 + exponente2) % 255
    return exponencial[exponenteT]

In [107]:
def GF_invers(a):
    exponente = logaritmica[a]
    return exponencial[255 - exponente]

# Comprobaciones

## Multiplicacion Polinomica

In [59]:
bien = True
for i in range(0, 255):
    for j in range(0, 255):
        bien = GF_product_p(i, j) == GF_product_p(j, i)
        if not bien:
            break
    if not bien:
        break
print(bien)

True


## Multiplicaciones Tablas

In [108]:
global exponencial
global logaritmica

GF_tables()

bien = True
for i in range(0, 255):
    for j in range(0, 255):
        bien = GF_product_t(i, j) == GF_product_t(j, i)
        if not bien:
            break
    if not bien:
        break
print(bien)

True


## Producto por su inverso

In [111]:
global exponencial
global logaritmica

GF_tables()

bien = True
for i in range(1, 255):
    print(i)
    bien = GF_product_t(i, GF_invers(i)) == 1
    if not bien:
        break
print(bien)

1


IndexError: list index out of range

# Pruebas

In [19]:
a = int(input("Insertar numero entero: "))
print(hex(a))
print(hex(GF_invers(a)))

print(hex(GF_product_p(0x02, 0x47)))
print(hex(GF_product_t(0x02, 0x47)))
print(hex(GF_product_p(0x47, 0x02)))

print()

inicio = timeit.default_timer()
resultado= GF_product_p(a,0x02)
final = timeit.default_timer()
print("Tiempo producto p(a,0x02) = " + str(final - inicio) + " resultado -> " + str(resultado))

inicio = timeit.default_timer()
resultado= GF_product_t(a,0x02)
final = timeit.default_timer()
print("Tiempo producto t(a,0x02) = " + str(final - inicio) + " resultado -> " + str(resultado))

print()

inicio = timeit.default_timer()
resultado= GF_product_p(a,0x03)
final = timeit.default_timer()
print("Tiempo producto p(a,0x03) = " + str(final - inicio) + " resultado -> " + str(resultado))

inicio = timeit.default_timer()
resultado= GF_product_t(a,0x03)
final = timeit.default_timer()
print("Tiempo producto t(a,0x03) = " + str(final - inicio) + " resultado -> " + str(resultado))

print()

inicio = timeit.default_timer()
resultado= GF_product_p(a,0x09)
final = timeit.default_timer()
print("Tiempo producto p(a,0x09) = " + str(final - inicio) + " resultado -> " + str(resultado))

inicio = timeit.default_timer()
resultado= GF_product_t(a,0x09)
final = timeit.default_timer()
print("Tiempo producto t(a,0x09) = " + str(final - inicio) + " resultado -> " + str(resultado))

print()

inicio = timeit.default_timer()
resultado= GF_product_p(a,0x0B)
final = timeit.default_timer()
print("Tiempo producto p(a,0x0B) = " + str(final - inicio) + " resultado -> " + str(resultado))

inicio = timeit.default_timer()
resultado= GF_product_t(a,0x0B)
final = timeit.default_timer()
print("Tiempo producto t(a,0x0B) = " + str(final - inicio) + " resultado -> " + str(resultado))

print()

inicio = timeit.default_timer()
resultado= GF_product_p(a,0x0D)
final = timeit.default_timer()
print("Tiempo producto p(a,0x0D) = " + str(final - inicio) + " resultado -> " + str(resultado))

inicio = timeit.default_timer()
resultado= GF_product_t(a,0x0D)
final = timeit.default_timer()
print("Tiempo producto t(a,0x0D) = " + str(final - inicio) + " resultado -> " + str(resultado))

print()

inicio = timeit.default_timer()
resultado= GF_product_p(a,0x0E)
final = timeit.default_timer()
print("Tiempo producto p(a,0x0E) = " + str(final - inicio) + " resultado -> " + str(resultado))

inicio = timeit.default_timer()
resultado= GF_product_t(a,0x0E)
final = timeit.default_timer()
print("Tiempo producto t(a,0x0E) = " + str(final - inicio) + " resultado -> " + str(resultado))

Insertar numero entero: 12
0xc
0x4c
0x8e
0x8e
0x8e

Tiempo producto p(a,0x02) = 0.00010674500026652822 resultado -> 24
Tiempo producto t(a,0x02) = 8.947900005296106e-05 resultado -> 24

Tiempo producto p(a,0x03) = 0.00010543200005486142 resultado -> 20
Tiempo producto t(a,0x03) = 9.05719998627319e-05 resultado -> 20

Tiempo producto p(a,0x09) = 0.00010599300003377721 resultado -> 108
Tiempo producto t(a,0x09) = 8.732500009500654e-05 resultado -> 108

Tiempo producto p(a,0x0B) = 0.00010744599967438262 resultado -> 116
Tiempo producto t(a,0x0B) = 8.512100021107472e-05 resultado -> 116

Tiempo producto p(a,0x0D) = 0.00010479100001248298 resultado -> 92
Tiempo producto t(a,0x0D) = 8.561200002077385e-05 resultado -> 92

Tiempo producto p(a,0x0E) = 0.00018907999992734403 resultado -> 72
Tiempo producto t(a,0x0E) = 8.699500040165731e-05 resultado -> 72
